In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/src'

/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/src


In [3]:
!pip install ipython-autotime
%load_ext autotime

time: 121 µs (started: 2021-06-03 00:16:22 +00:00)


In [4]:
import torch
device = torch.device("cuda")
device

device(type='cuda')

time: 2.06 s (started: 2021-06-03 00:16:26 +00:00)


In [6]:
%cd '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/src'

/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/src
time: 2.9 ms (started: 2021-06-03 00:16:49 +00:00)


In [7]:
from data.generator import DataGenerator as DataGenerator_orig, Tokenizer as Tokenize_orig



time: 315 ms (started: 2021-06-03 00:16:52 +00:00)


In [8]:
import argparse
import os
import string

import cv2
import h5py
import numpy as np
import torch
import torchvision.transforms as T
from data import preproc as pp, evaluation
from data.generator import DataGenerator as DataGenerator_orig, Tokenizer as Tokenize_orig
from data.reader import Dataset
from engine import single_image_inference, LabelSmoothing, run_epochs, get_memory
from network.model import make_model
from network.predictors import Predictor

time: 69 ms (started: 2021-06-03 00:16:55 +00:00)


In [11]:
  def subsequent_masking(x):
      # x: (batch_size, seq_len - 1)
      batch_size, seq_len = x.size()
      subsequent_mask = np.triu(np.ones(shape=(seq_len, seq_len)), k=1).astype('bool')
      subsequent_mask = torch.tensor(subsequent_mask).to(x.device)
      subsequent_mask = subsequent_mask.unsqueeze(0).expand(batch_size, seq_len, seq_len)
      return subsequent_mask

  PAD_TOKEN_ID = 0

  PAD_TOKEN_INDEX = 0


  PAD_TOKEN_INDEX = 0
  vocab_length = 99
  def pad_masking(x, target_len):
      # x: (batch_size, seq_len)
      batch_size, seq_len = x.size()
      padded_positions = x == PAD_TOKEN_INDEX  # (batch_size, seq_len)
      pad_mask = padded_positions.unsqueeze(1).expand(batch_size, target_len, seq_len)
      return pad_mask

  def get_memory(model, imgs):
      x = model.conv(model.get_feature(imgs))
      bs,_,H, W = x.shape
      pos = torch.cat([
              model.col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
              model.row_embed[:H].unsqueeze(1).repeat(1, W, 1),
          ], dim=-1).flatten(0, 1).unsqueeze(0)

      

      #print ("x shape : ",x.shape)
      #print ("pos shape : ",pos.shape)

      memory =  model.transformer.encoder(pos + 0.1 * x.flatten(2).permute(0, 2, 1))
      #print ("Memory shape from get_memmory:",memory.shape)
      return memory

time: 17.6 ms (started: 2021-06-03 00:17:15 +00:00)


In [10]:
  import os
  import re
  root_path = "/content/drive/MyDrive/handwritten-text-recognition-for-apache-mxnet/images/"
  file_list = os.listdir(root_path)
  file_list.sort(key=lambda f: int(re.sub('\D', '', f)))
  file_list

['line_image_0.png',
 'line_image_1.png',
 'line_image_2.png',
 'line_image_3.png',
 'line_image_4.png',
 'line_image_5.png',
 'line_image_6.png',
 'line_image_7.png']

time: 254 ms (started: 2021-06-03 00:17:09 +00:00)


In [12]:
input_size = (1024, 128, 1)
max_text_length = 128
charset_base = string.printable[:95]
tokenizer_orig = Tokenize_orig(chars=charset_base, max_text_length=max_text_length)   

time: 2.23 ms (started: 2021-06-03 00:17:19 +00:00)


In [13]:
  
  source_path = '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/data/iam.hdf5'
  target_path = '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/output/iam/checkpoint_weights.pt'
  model = make_model(tokenizer_orig.vocab_size, hidden_dim=256, nheads=4,
  num_encoder_layers=4, num_decoder_layers=4)

  #print ("Memory shape from get_memmory:",memory.shape)
  device = torch.device("cuda")
  model.to(device)
  transform = T.Compose([
          T.ToTensor()])
          

  if os.path.exists(target_path):
      model.load_state_dict(torch.load(target_path))  
      print ("Model target Path:",target_path)          
  else:            
    print('No model checkpoint found')
    
  model.eval()
  predicts = []
  gt = []
  print ("source_path :",source_path)
  with torch.no_grad():
     for i,image_file_name in enumerate(file_list):

        image_full_path = root_path+image_file_name
        #print ("image_full_path",image_full_path)
        img = pp.preprocess(image_full_path, input_size=input_size)
        
        #making image compitable with resnet
        img = np.repeat(img[..., np.newaxis],3, -1)
        img = pp.normalization(img)
        #print ("src shape :",src.shape)
        img = transform(img)    
        imgs = img.unsqueeze(0).float().to(device)
        memory = get_memory(model,imgs)

        predictor = Predictor(
            model=model,tokenizer=tokenizer_orig
        )

        for index, candidate in enumerate(predictor.predict_one(memory, num_candidates=1)):
              candidate = candidate.replace('EOS','')
              #print(f'Candidate {index} : {candidate}')
              predicts.append(candidate)

  predicts = list(map(lambda x : x.replace('SOS','').replace('EOS',''),predicts))
  gt = list(map(lambda x : x.replace('SOS','').replace('EOS',''),gt))
  


Model target Path: /content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/output/iam/checkpoint_weights.pt
source_path : /content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/data/iam.hdf5
time: 18.2 s (started: 2021-06-03 00:17:22 +00:00)


In [14]:
predicts

["A subomariner ' s wife needed to be spared as mud",
 'as possible . Anyloy the Parsilal affair was far',
 'too fresh in both their minds tobe a comfortable',
 'subject for discussion . " Hs a command , " he said . ',
 'We can do with the ertra money . Jtll sist about',
 'pay faculs sclool fees . " what shall wedo',
 "about this housel - and fill ' s saod ( and . ' ",
 'how 3 hate the Nany ) ']

time: 3.87 ms (started: 2021-06-03 00:17:47 +00:00)


## Including Spell Check

In [16]:
%cd "/content/drive/MyDrive/spelling-correction/src"

/content/drive/MyDrive/spelling-correction/src
time: 2.45 ms (started: 2021-06-03 00:18:17 +00:00)


In [17]:
import os
import datetime
import string

# define parameters
source = "iam"
mode = "luong"
epochs = 1000
batch_size = 64

# define paths
data_path = os.path.join("/content/drive/MyDrive/spelling-correction", "data")
source_path = os.path.join(data_path, f"{source}.txt")

output_path = os.path.join("/content/drive/MyDrive/spelling-correction", "output", source, mode)
target_path = os.path.join(output_path, "checkpoint_weights.hdf5")
os.makedirs(output_path, exist_ok=True)

# define number max of chars per line and list of valid chars
max_text_length = 128
charset_base = string.printable[:95]
charset_special = """ÀÁÂÃÄÅÇÈÉÊËÌÍÎÏÑÒÓÔÕÖÙÚÛÜÝàáâãäåçèéêëìíîïñòóôõöùúûüý"""

print("output", output_path)
print("target", target_path)
print("charset:", charset_base + charset_special)

output /content/drive/MyDrive/spelling-correction/output/iam/luong
target /content/drive/MyDrive/spelling-correction/output/iam/luong/checkpoint_weights.hdf5
charset: 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ ÀÁÂÃÄÅÇÈÉÊËÌÍÎÏÑÒÓÔÕÖÙÚÛÜÝàáâãäåçèéêëìíîïñòóôõöùúûüý
time: 692 ms (started: 2021-06-03 00:18:20 +00:00)


In [18]:
from data2.generator2 import DataGenerator as DataGenerator2

dtgen = DataGenerator2(source=source_path,
                      batch_size=64,
                      charset=(charset_base + charset_special),
                      max_text_length=128)

print(f"Train sentences: {dtgen.size['train']}")
print(f"Validation sentences: {dtgen.size['valid']}")
print(f"Test sentences: {dtgen.size['test']}")

Train sentences: 145314
Validation sentences: 16145
Test sentences: 32
time: 7.38 s (started: 2021-06-03 00:18:25 +00:00)


In [19]:
from data2 import preproc as pp, evaluation as ev
from tool.seq2seq import Seq2SeqAttention
from tool.transformer import Transformer
target_path ='/content/drive/MyDrive/spelling-correction/output/iam/luong/checkpoint_weights.hdf5'
spell_check_model = Seq2SeqAttention(dtgen.tokenizer,
                             mode,
                             units=512,
                             dropout=0.2,
                             stop_tolerance=20,
                             reduce_tolerance=15)

spell_check_model.compile(learning_rate=0.001)
#spell_check_model.summary(output_path, "summary.txt")

# get default callbacks list and load checkpoint weights file (HDF5) if exists 
spell_check_model.load_checkpoint(target=target_path)

callbacks = spell_check_model.get_callbacks(logdir=output_path, checkpoint=target_path, verbose=1)

time: 13.3 s (started: 2021-06-03 00:18:37 +00:00)


In [20]:
from data2 import preproc as pp


time: 935 µs (started: 2021-06-03 00:19:45 +00:00)


In [21]:
input_list =predicts

def next_test_batch(input_list):
  inputs = dtgen.prepare_sequence(sentences=input_list, sos=True, eos=True)
  yield [inputs]

predicts_spell_correct = spell_check_model.predict(x = next_test_batch(input_list))
predicts_spell_correct = [pp.text_standardize(x) for x in predicts_spell_correct]


time: 10.1 s (started: 2021-06-03 00:19:49 +00:00)


In [22]:
predicts_spell_correct

["A submariner ' s wife needed to be spared as much",
 'as possible . Anyway the Parsifal affair was far',
 'too fresh in both their minds to be a comfortable',
 'subject of discussion . " He command , " he said . " Or .',
 'We can do with the ertra money . All siti about',
 'pay faculs closely . " I shall we do',
 "about this house - and fill ' s sard - and . '",
 'how that each , many -']

time: 2.91 ms (started: 2021-06-03 00:20:08 +00:00)
